In [1]:
import PyPDF2
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document


In [2]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text
def split_text(text, chunk_size=500):
    return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]


In [3]:
llm = OllamaLLM(model="llama3.2", server_url="http://localhost:11434")  

In [4]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = Chroma(persist_directory="./vectorstore", embedding_function=embedding_model)


/home/bigdata/miniconda3/envs/rag/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
pdf_path = "PDF.pdf"

page_content = extract_text_from_pdf(pdf_path)

chunks = split_text(page_content)
print(f"Number of chunks: {len(chunks)}")


Number of chunks: 57


In [6]:
documents = [Document(page_content=chunk) for chunk in chunks]
vectorstore.add_documents(documents)
print("Documents added to the vector store.")


Documents added to the vector store.


In [7]:
prompt = ChatPromptTemplate.from_template(
    template="Usa el contexto a continuación para responder la pregunta del usuario:\n\n{context}\n\nPregunta: {question}\nRespuesta:"
)


In [8]:
retriever = vectorstore.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)


In [9]:
query = "¿Cuál es el tema principal del documento?"
response = qa_chain.run(query)
print("Respuesta:", response)


/tmp/ipykernel_21129/1288733369.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)


Respuesta: El tema principal del documento parece ser la historia y el contexto de la piratería en Inglaterra durante la Edad Moderna, destacando las diferencias con España y cómo se reconocían e incentivaban los innovadores y personas que aportaban nuevas ideas al estado. El texto menciona a varios científicos y autores que escribieron sobre esta época, como Agrícola, Biringuccio, Lastanosa, Lobato y Turriano, y destaca la importancia de la innovación y el talento en resolver problemas sociales y económicos.
